# NLP 3A - Kaggle Competition

## 0 - Imports

In [47]:
import json

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from tqdm import tqdm
import re
import string
import nltk
from sklearn.utils import shuffle
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler

stop_words = stopwords.words('english')

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from sklearn.metrics import accuracy_score


import matplotlib.pyplot as plt
import seaborn as sns

In [48]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Louis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## I - Baseline code

In [49]:
with open('test.json', 'r') as file:
    data = json.load(file)

f_data = {}
for k, v in data.items():
    f_data[k] = v[:95] # Ok, en gros f_date = {'Sport': [Liste des sequences de theme sport],...}

processed_data = {
    'text': [], 'target': []
}
for k, v in f_data.items():
    processed_data['text'] += v
    processed_data['target'] += [k] * len(v) # {'text': [Liste des phrases], 'target': [Liste des labels]}

df = pd.DataFrame.from_dict(processed_data)
df.head()

,text,target
0,The team scored a last-minute goal to win the ...,Sports
1,The tennis player served an ace to win the match.,Sports
2,"The basketball game went into overtime, but th...",Sports
3,The marathon runner broke the world record wit...,Sports
4,The soccer match ended in a draw after a thril...,Sports


In [50]:
with open('train.json', 'r') as file:
    data = json.load(file)

f_data = {}
for k, v in data.items():
    f_data[k] = v[:95] # Ok, en gros f_date = {'Sport': [Liste des sequences de theme sport],...}

processed_data = {
    'text': [], 'target': []
}
for k, v in f_data.items():
    processed_data['text'] += v
    processed_data['target'] += [k] * len(v) # {'text': [Liste des phrases], 'target': [Liste des labels]}

df_test = pd.DataFrame.from_dict(processed_data)
df_test.head()

,text,target
0,The mayor announced a new initiative to improv...,Politics
1,The senator is facing criticism for her stance...,Politics
2,The upcoming election has sparked intense deba...,Politics
3,Regular exercise and a balanced diet are key t...,Health
4,The World Health Organization has issued new g...,Health


In [52]:
# X_train, X_test, y_train, y_test = train_test_split(df['text'].values, df['target'].values, test_size=0,
#                                                    random_state=123, stratify=df['target'].values)

X_train = df['text'].values
y_train = df['target'].values
X_train, y_train = shuffle(X_train, y_train)
with open('test_shuffle.txt', 'w') as file:
    for item in X_train.tolist():
        file.write(item + "\n")

with open('y_test_shuffle.txt', 'w') as file:
    for item in y_train.tolist():
        file.write(item + "\n")

y_test = df_test['target'].values
X_test = df_test['text'].values

# II - Models

### 1 - TfidfVectorizer + Random Forest

In [53]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_train_vectors = tfidf_vectorizer.fit_transform(X_train)
tfidf_test_vectors = tfidf_vectorizer.transform(X_test)

classifier = RandomForestClassifier() # RF baseline model
classifier.fit(tfidf_train_vectors, y_train)

y_pred = classifier.predict(tfidf_test_vectors)


with open('y_pred_shuffle.txt', 'w') as file:
    for item in y_pred.tolist():
        file.write(item + "\n")
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
# print(classification_report(y_test, y_pred))

Accuracy: 0.42


### 2 - Bert

In [54]:
num_unique_labels = df['target'].nunique()
print(f'The number of unique labels is: {num_unique_labels}')

The number of unique labels is: 12


In [55]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=12)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# III - Results

In [19]:
results_df = pd.DataFrame({
    'Sentence': X_test,
    'Actual Target': y_test,
    'Predicted Target': y_pred
})
results_df.head(5)

,Sentence,Actual Target,Predicted Target
0,Politics,The mayor announced a new initiative to improv...,Entertainment
1,Politics,The senator is facing criticism for her stance...,Entertainment
2,Politics,The upcoming election has sparked intense deba...,Entertainment
3,Health,Regular exercise and a balanced diet are key t...,Technology
4,Health,The World Health Organization has issued new g...,Technology


# II - Label to submission

In [18]:
with open('y_pred_shuffle.txt', 'r') as file:
    datas = file.readlines()

with open('y_test_shuffle_for_kaggle.txt', 'w') as file:
    i = 0
    file.write(f"ID,Usage,Label\n")
    for data in datas:
        file.write(f"{i},Private,{data}") if i % 2 == 1 else file.write(f"{i},Public,{data}")
        i += 1

with open('y_test_shuffle.txt', 'r') as file:
    datas = file.readlines()

with open('y_test_shuffle_for_kaggle.txt', 'w') as file:
    i = 0
    file.write(f"ID,Usage,Label\n")
    for data in datas:
        file.write(f"{i},Private,{data}")
        i += 1